In [8]:
!pip install gradio
!pip install torch

In [16]:
import gradio as gr
import torch
import torchvision.transforms as transforms
from PIL import Image
import torch
import torch.nn as nn
import torch.nn.functional as F

class ImprovedNet(nn.Module):
    def __init__(self, input_channels=3, input_size=32):
        super(ImprovedNet, self).__init__()
        self.conv1 = nn.Conv2d(input_channels, 64, 3, padding=1)
        self.bn1 = nn.BatchNorm2d(64)
        self.conv2 = nn.Conv2d(64, 128, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(128)
        self.conv3 = nn.Conv2d(128, 256, 3, padding=1)
        self.bn3 = nn.BatchNorm2d(256)
        self.conv4 = nn.Conv2d(256, 512, 3, padding=1)
        self.bn4 = nn.BatchNorm2d(512)
        self.pool = nn.MaxPool2d(2, 2)

        # Update the input size calculation for the fully connected layers
        fc_input_size = 512 * (input_size // 16) * (input_size // 16)

        self.fc1 = nn.Linear(fc_input_size, 1024)
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, 10)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        x = self.pool(F.relu(self.bn2(self.conv2(x))))
        x = self.pool(F.relu(self.bn3(self.conv3(x))))
        x = self.pool(F.relu(self.bn4(self.conv4(x))))
        x = x.view(x.size(0), -1)  # Flatten the output before the fully connected layers
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        return x

model = torch.load('modell.pt', map_location=torch.device('cpu'))
loaded_model = ImprovedNet()
loaded_model.load_state_dict(model.state_dict())

classes=['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

def preprocess(img):
    img = Image.fromarray(img.astype('uint8'), 'RGB')
    transform = transforms.Compose([
        transforms.Resize((32, 32)),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])
    img = transform(img)
    img = img.unsqueeze(0)  # Add batch dimension
    return img

# Make predictions
def predict(img):
    img = preprocess(img)
    output = model(img)
    _, predicted_class = torch.max(output, 1)
    predicted_class_number = torch.argmax(output).item()
    predicted_class_name = classes[predicted_class_number]
    prediction_dict = {predicted_class_name: 0.9}
    return prediction_dict
# Return the dictionary instead of a tuple

# Gradio Interface
iface = gr.Interface(
    fn=predict,
    inputs=gr.Image(),
    outputs="label",
    live=True
)

# Launch the Gradio Interface
iface.launch()


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://d72c8eb1fc9703a595.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
